read train data

In [81]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn import cross_validation

In [82]:
data = pd.read_csv('crx_data_train_x.csv', header=None, na_values='?')
y = pd.read_csv('crx_data_train_y.csv', header=None, na_values='?')

merge and drop id column

In [83]:
data = pd.merge(data, y, on=0)
data = pd.DataFrame(data)
data.drop(data.columns[[0]], axis=1, inplace=True)

name columns

In [84]:
data.columns = ['A' + str(i) for i in range(1, len(data.columns))]+['class']
data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,class
0,b,35.58,0.75,u,g,k,v,1.500,f,f,0,t,g,231,0,-
1,b,34.92,2.50,u,g,w,v,0.000,t,f,0,t,g,239,200,+
2,b,NaN,4.00,y,p,i,v,0.085,f,f,0,t,g,411,0,-
3,a,46.08,3.00,u,g,c,v,2.375,t,t,8,t,g,396,4159,+
4,a,23.58,0.83,u,g,q,v,0.415,f,t,1,t,g,200,11,-


In [85]:
data.count(axis=0)

A1       476
A2       476
A3       483
A4       479
A5       479
A6       477
A7       477
A8       483
A9       483
A10      483
A11      483
A12      483
A13      483
A14      477
A15      483
class    483
dtype: int64

In [86]:
data = data.fillna(data.median(axis=0), axis=0)

In [87]:
data.count(axis=0)

A1       476
A2       483
A3       483
A4       479
A5       479
A6       477
A7       477
A8       483
A9       483
A10      483
A11      483
A12      483
A13      483
A14      483
A15      483
class    483
dtype: int64

In [88]:
data.describe(include=[object])

,A1,A4,A5,A6,A7,A9,A10,A12,A13,class
count,476,479,479,477,477,483,483,483,483,483
unique,2,3,3,14,8,2,2,2,3,2
top,b,u,g,c,v,t,f,f,g,-
freq,329,364,364,91,279,261,270,253,439,261


In [89]:
data_describe = data.describe(include=[object])
categorical_columns = data.describe(include=[object]).columns.values
numerical_columns = [c for c in data.columns if data[c].dtype.name != 'object']
for c in categorical_columns:
    data[c] = data[c].fillna(data_describe[c]['top'])

In [90]:
data[categorical_columns].describe()

,A1,A4,A5,A6,A7,A9,A10,A12,A13,class
count,483,483,483,483,483,483,483,483,483,483
unique,2,3,3,14,8,2,2,2,3,2
top,b,u,g,c,v,t,f,f,g,-
freq,336,368,368,97,285,261,270,253,439,261


In [91]:
binary_columns = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]
for c in binary_columns:
    top = data_describe[c]['top']
    top_items = data[c] == top
    data.loc[top_items, c] = 0
    data.loc[np.logical_not(top_items), c] = 1

In [92]:
data[binary_columns].describe()

,A1,A9,A10,A12,class
count,483,483,483,483,483
unique,2,2,2,2,2
top,0,0,0,0,0
freq,336,261,270,253,261


In [93]:
data_nonbinary = pd.get_dummies(data[nonbinary_columns])

numerical data standardization

In [98]:
data_numerical = data[numerical_columns]
data_numerical_normal_1 = (data_numerical - data_numerical.mean()) / data_numerical.std()
data_numerical_normal_2 = (data_numerical - data_numerical.mean()) / (data_numerical.max()-data_numerical.min())

complite data 

In [99]:
data = pd.concat((data_numerical_normal_1, data[binary_columns], data_nonbinary), axis=1)
data = pd.DataFrame(data, dtype=float)
data.columns.values

array(['A2', 'A3', 'A8', 'A11', 'A14', 'A15', 'A1', 'A9', 'A10', 'A12',
       'class', 'A4_l', 'A4_u', 'A4_y', 'A5_g', 'A5_gg', 'A5_p', 'A6_aa',
       'A6_c', 'A6_cc', 'A6_d', 'A6_e', 'A6_ff', 'A6_i', 'A6_j', 'A6_k',
       'A6_m', 'A6_q', 'A6_r', 'A6_w', 'A6_x', 'A7_bb', 'A7_dd', 'A7_ff',
       'A7_h', 'A7_j', 'A7_n', 'A7_v', 'A7_z', 'A13_g', 'A13_p', 'A13_s'], dtype=object)

split data

In [100]:
X = data.drop('class', axis=1)
y = data['class']

Feature importance